In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
#from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = hps.data.dict.text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

## Pre-Trained CSS10 Es

In [2]:
hps = utils.get_hparams_from_file("configs/css10_de_char_base.json")

In [3]:
net_g = SynthesizerTrn(
    len(hps.data.dict.symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model).cuda()
_ = net_g.eval()

_ = utils.load_checkpoint("G_24000.pth", net_g, None)

In [38]:
stn_tst = get_text("Lange, ehe der Krieg Japans mit Rußland begann, hörte Hanake in ihrem Hause am Biwasee von Freunden und Freundinnen,", hps)
# stn_tst = get_text("Hallo meine Freunde", hps)
# stn_tst = get_text("Hello my friends", hps)
# stn_tst = get_text("Hallo Freunde", hps)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    sid = torch.LongTensor([0]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))
# ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=True))